In [1]:
import rasterio
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AdaptiveAvgPool2d
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torchvision import models
from sklearn.metrics import accuracy_score
import torch.optim as optim
from model_pre_functions import *

In [ ]:
X_target_train= np.load('/Volumes/ksankaran/sen12ms_subsample2/X_target_train.npy')
X_target_test  = np.load('/Volumes/ksankaran/sen12ms_subsample2/X_target_test.npy')
y_target_test = np.load('/Volumes/ksankaran/sen12ms_subsample2/y_target_test.npy')
y_target_train = np.load('/Volumes/ksankaran/sen12ms_subsample2/y_target_train.npy')

In [ ]:
X_source = np.load('/Volumes/ksankaran/sen12ms_subsample2/X_source.npy')
y_source = np.load('/Volumes/ksankaran/sen12ms_subsample2/y_source.npy')
X_source.shape,y_source.shape

In [6]:
y_target_test = y_target_test-1
y_target_train = y_target_train -1
y_source = y_source 

### Obtain the pretrained VGG model

In [9]:
# load the pretraind and un-pretrained VGG models
pre_vgg = models.vgg16_bn(pretrained=True)
# resize the first layer of the pretrained vgg model
pre_vgg.features[0] = Conv2d(13,64,kernel_size=(3,3),stride=(1,1),padding=(1,1))

# concatenate weights of channels
temp = torch.cat((pre_vgg.state_dict()['features.0.weight'],pre_vgg.state_dict()['features.0.weight'],
               pre_vgg.state_dict()['features.0.weight'],pre_vgg.state_dict()['features.0.weight'],
               pre_vgg.state_dict()['features.0.weight']),1)
temp = temp[:,0:13,:,:]
pre_vgg.state_dict()['features.0.weight'] = temp
#print(pre_vgg)

In [15]:
d2 = get_dict(y_source)

In [29]:
d2

{0: 4, 5: 5, 6: 6, 7: 8, 9: 9, 2: 1, 4: 3, 1: 0, 3: 2, 8: 7}

In [17]:
train_x,train_y = prepare_data(X_source,y_source)
train_x.shape,train_y.shape

(torch.Size([300, 13, 256, 256]), torch.Size([300]))

In [21]:
x_train,y_train = extract_features(train_x,train_y, batch_size=8)

100%|██████████| 38/38 [03:39<00:00,  5.79s/it]


In [26]:
val_x,val_y = prepare_data(X_target_train,y_target_train)
val_x.shape,val_y.shape

(torch.Size([2673, 13, 256, 256]), torch.Size([2673]))

In [27]:
x_val,y_val = extract_features(val_x,val_y, batch_size=8)

100%|██████████| 335/335 [33:22<00:00,  5.98s/it]


In [19]:
test_x,test_y = prepare_data(X_target_test,y_target_test)
test_x.shape,test_y.shape

(torch.Size([27, 13, 256, 256]), torch.Size([27]))

In [20]:
x_test,y_test = extract_features(test_x,test_y, batch_size=8)

100%|██████████| 4/4 [00:24<00:00,  6.23s/it]


### Train a model on the source data.

In [14]:
model = pre_vgg
model.classifier[6] = Sequential(
                      Linear(4096, 10))

In [22]:
# specify loss function (categorical cross-entropy)
criterion = CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.0005)

get the data from source for training

In [22]:
train_model(x_train,y_train, model, criterion, optimizer,
                n_epochs=2,
                batch_size=8)

 50%|█████     | 1/2 [00:19<00:19, 19.52s/it]

epoch: 	 1 	 training loss: 	 1.3111899887260638


100%|██████████| 2/2 [00:37<00:00, 18.91s/it]

epoch: 	 2 	 training loss: 	 1.0210108176658028


VGG(
  (features): Sequential(
    (0): Conv2d(13, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 25

In [23]:
#torch.save(model,'/Volumes/ksankaran/sen12ms_subsample2/models/model_vgg_trained.pt')

In [25]:
pred_acc(x_train,y_train,model,batch_size=8)

100%|██████████| 38/38 [00:04<00:00,  8.86it/s]


0.555921052631579

In [28]:
pred_acc(x_val,y_val,model,batch_size=8)

100%|██████████| 335/335 [00:38<00:00,  8.78it/s]


0.13880597014925372

In [31]:
for param in model.features[0:42].parameters():
    param.requires_grad = False
for param in model.features[40:43].parameters():
    param.requires_grad = True
for parameter in model.classifier.parameters():
    parameter.requires_grad = True

In [32]:
train_model(x_val,y_val, model, criterion, optimizer,
                n_epochs=2,
                batch_size=8)

 50%|█████     | 1/2 [02:39<02:39, 159.93s/it]

epoch: 	 1 	 training loss: 	 0.8307030611296198


100%|██████████| 2/2 [05:19<00:00, 159.75s/it]

epoch: 	 2 	 training loss: 	 0.7622745642466332


VGG(
  (features): Sequential(
    (0): Conv2d(13, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 25

In [38]:
#torch.save(model,'/Volumes/ksankaran/sen12ms_subsample2/models/model_vgg_tuned.pt')

In [7]:
#model = torch.load('/Volumes/ksankaran/sen12ms_subsample2/models/model_vgg_tuned.pt')

In [34]:
pred_acc(x_val,y_val,model,batch_size=8)

100%|██████████| 335/335 [00:32<00:00, 10.36it/s]


0.7600746268656716

In [37]:
pred_acc(x_test,y_test,model,batch_size=8)

100%|██████████| 4/4 [00:00<00:00,  5.60it/s]


0.625

In [23]:
train_model(x_test,y_test, model, criterion, optimizer,
                n_epochs=2,
                batch_size=8)

 50%|█████     | 1/2 [00:06<00:06,  6.12s/it]

epoch: 	 1 	 training loss: 	 0.7008608654141426


100%|██████████| 2/2 [00:08<00:00,  4.05s/it]

epoch: 	 2 	 training loss: 	 0.4831443727016449


VGG(
  (features): Sequential(
    (0): Conv2d(13, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 25

In [41]:
pred_acc(x_test,y_test,model,batch_size=8)

100%|██████████| 4/4 [00:00<00:00, 10.65it/s]


0.78125

In [42]:
pred_acc(x_train,y_train,model,batch_size=8)

100%|██████████| 38/38 [00:03<00:00,  9.87it/s]


0.08223684210526316